**import libraries**

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

**Load Data**

In [2]:
high_use = pd.read_excel('C:/Users/bgc/Desktop/input_high_use.xlsx')
high_use.head(10)

C:\Users\bgc\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ITEM_ITEM_ID,COD_ITEM,PRICE,NUM_MONTH_USE_SPMUS,QTY_NET_USE_SPMUS
0,630992.0,401052910027,29000000.0,139801.0,0.0
1,630992.0,401052910027,29000000.0,139802.0,0.0
2,630992.0,401052910027,29000000.0,139803.0,0.0
3,630992.0,401052910027,29000000.0,139804.0,0.0
4,630992.0,401052910027,29000000.0,139805.0,2.0
5,630992.0,401052910027,29000000.0,139806.0,6.0
6,630992.0,401052910027,29000000.0,139807.0,0.0
7,630992.0,401052910027,29000000.0,139808.0,0.0
8,630992.0,401052910027,29000000.0,139809.0,0.0
9,630992.0,401052910027,29000000.0,139810.0,0.0


**preparition**

In [3]:
#max available data
numtotal_periods = 48
# last 
last_month = str(max(high_use['NUM_MONTH_USE_SPMUS']))
print(last_month)
num_month = int(last_month[4:6])
print(num_month)
forecast_horizen = 12 - num_month
forecast_horizen

140109.0
9


3

**Pivot Table**

In [4]:
pivot_table = high_use.pivot_table(values = 'QTY_NET_USE_SPMUS' , index = 'ITEM_ITEM_ID' , columns = 'NUM_MONTH_USE_SPMUS' , 
                     aggfunc='sum',fill_value = 0 , margins = False)
pivot_table2 = pivot_table.copy()

pivot_table['sum_all'] = pivot_table2.sum(axis = 1)
pivot_table['avg_real'] = pivot_table2.mean(axis = 1)
pivot_table

NUM_MONTH_USE_SPMUS,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,...,140102.0,140103.0,140104.0,140105.0,140106.0,140107.0,140108.0,140109.0,sum_all,avg_real
ITEM_ITEM_ID,,,,,,,,,,,,,,,,,,,,,
630992.0,0,0.0,0,0,2,6,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,29.0,0.644444
631022.0,0,0.0,2,0,0,3,0.0,7,0.0,0,...,0,0,1,0,0,0,0,0.0,21.0,0.466667
631057.0,0,0.0,0,4,0,4,4.0,0,4.0,0,...,0,0,6,9,6,0,0,0.0,81.0,1.800000
631079.0,0,0.0,0,0,2,0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,1.0,6.0,0.133333
631080.0,0,0.0,0,0,0,0,0.0,0,4.0,1,...,0,0,0,0,1,0,0,0.0,23.0,0.511111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862202.0,0,0.0,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0.0,10.0,0.222222
1862908.0,0,0.0,0,0,0,0,0.0,0,2.0,0,...,0,1,10,5,0,0,24,0.0,70.0,1.555556
1864150.0,0,0.0,0,0,0,0,0.0,0,10.0,0,...,0,0,0,0,0,0,0,5.0,25.0,0.555556


**Define Category(Daste)**

In [5]:
Category = [-(math.inf), 3, 5.5, 7.5, 11.5, 15.5, 22.5, 32.5, 52.5, 94.5, 225.5, math.inf]
Category

[-inf, 3, 5.5, 7.5, 11.5, 15.5, 22.5, 32.5, 52.5, 94.5, 225.5, inf]

In [6]:
daste = []
for qty in pivot_table['sum_all']:
    for i in range(len(Category)):
        if qty<= Category[i] :
            daste.append(Category[i-1])
            break
                        
pivot_table['daste'] = daste
pivot_table

NUM_MONTH_USE_SPMUS,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,...,140103.0,140104.0,140105.0,140106.0,140107.0,140108.0,140109.0,sum_all,avg_real,daste
ITEM_ITEM_ID,,,,,,,,,,,,,,,,,,,,,
630992.0,0,0.0,0,0,2,6,0.0,0,0.0,0,...,0,0,0,0,0,0,0.0,29.0,0.644444,22.5
631022.0,0,0.0,2,0,0,3,0.0,7,0.0,0,...,0,1,0,0,0,0,0.0,21.0,0.466667,15.5
631057.0,0,0.0,0,4,0,4,4.0,0,4.0,0,...,0,6,9,6,0,0,0.0,81.0,1.800000,52.5
631079.0,0,0.0,0,0,2,0,0.0,0,0.0,0,...,0,0,0,0,0,0,1.0,6.0,0.133333,5.5
631080.0,0,0.0,0,0,0,0,0.0,0,4.0,1,...,0,0,0,1,0,0,0.0,23.0,0.511111,22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1862202.0,0,0.0,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0,0,0.0,10.0,0.222222,7.5
1862908.0,0,0.0,0,0,0,0,0.0,0,2.0,0,...,1,10,5,0,0,24,0.0,70.0,1.555556,52.5
1864150.0,0,0.0,0,0,0,0,0.0,0,10.0,0,...,0,0,0,0,0,0,5.0,25.0,0.555556,22.5


In [7]:
pivot_table.describe()

NUM_MONTH_USE_SPMUS,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,...,140103.0,140104.0,140105.0,140106.0,140107.0,140108.0,140109.0,sum_all,avg_real,daste
count,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,...,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.000000,7223.0
mean,2.430015,5.130569,3.807559,5.382666,3.739997,4.253634,6.629960,4.899903,4.220199,4.885782,...,4.616226,3.840233,7.000277,5.937145,3.362869,4.860446,5.199419,201.032696,4.467393,-inf
std,26.056599,48.278292,26.688352,83.242359,31.733677,43.038736,89.070741,49.326745,44.378146,44.165321,...,48.317562,44.083305,90.253726,95.593653,25.114266,45.610222,65.850633,1539.240351,34.205341,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.066667,-inf
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.244444,7.5
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.000000,0.555556,22.5
75%,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.000000,1.711111,52.5
max,1000.000000,2131.000000,794.000000,5000.000000,1804.000000,1990.000000,5500.000000,2000.000000,2800.000000,2000.000000,...,2500.000000,2000.000000,4000.000000,5000.000000,945.000000,2000.000000,3000.000000,71000.000000,1577.777778,225.5


**KMeans Clustering**

In [8]:
daste_group = pivot_table.groupby('daste')['sum_all'].agg(np.sum).to_frame()
daste_ha = list(daste_group.index)
clustered_list = []
for group in daste_ha:
    cluster_group = []
    table_daste = pivot_table[pivot_table['daste'] == group]
    X = table_daste.iloc[: ,33:46 ].values
    kmeans = KMeans(n_clusters=2, init='k-means++', max_iter=300, n_init=10, random_state=0)
    y = kmeans.fit_predict(X)
    table_daste['cluster'] = y
    clustered_list.append(table_daste)
    
    
    
    
    

C:\Users\bgc\AppData\Local\Temp\ipykernel_11308\4127551049.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_daste['cluster'] = y
C:\Users\bgc\AppData\Local\Temp\ipykernel_11308\4127551049.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_daste['cluster'] = y
C:\Users\bgc\AppData\Local\Temp\ipykernel_11308\4127551049.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

**Create Final Clusters**

In [9]:
result_kmeans = pd.concat(clustered_list)
result_kmeans

NUM_MONTH_USE_SPMUS,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,...,140104.0,140105.0,140106.0,140107.0,140108.0,140109.0,sum_all,avg_real,daste,cluster
ITEM_ITEM_ID,,,,,,,,,,,,,,,,,,,,,
632927.0,0,0.0,0,0,0,0,0.0,1,0.0,0,...,0,0,0,0,0,0.0,3.0,0.066667,-inf,0
639349.0,0,0.0,0,0,0,0,0.0,0,1.0,0,...,0,0,0,0,0,0.0,3.0,0.066667,-inf,0
640144.0,1,0.0,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0,0.0,3.0,0.066667,-inf,0
642304.0,0,0.0,0,0,0,0,0.0,0,1.0,0,...,0,0,0,0,0,0.0,3.0,0.066667,-inf,0
642900.0,0,1.0,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0,0.0,3.0,0.066667,-inf,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766550.0,0,15.0,0,0,10,0,20.0,0,30.0,0,...,5,0,0,10,0,0.0,246.0,5.466667,225.5,0
1774398.0,0,1.0,12,48,35,35,65.0,38,106.0,26,...,3,148,12,29,51,65.0,2645.0,58.777778,225.5,0
1784784.0,0,14.0,0,60,30,0,20.0,0,12.0,0,...,0,0,16,0,0,20.0,421.0,9.355556,225.5,0


In [10]:
final_cluster = []
for i in range(result_kmeans.shape[0]):
    d =str(result_kmeans['daste'].values[i])
    cl = str(result_kmeans['cluster'].values[i])
    f_cluster = d + '_' + cl
    final_cluster.append(f_cluster)
    
result_kmeans['final_cluster'] = final_cluster   
result_kmeans

NUM_MONTH_USE_SPMUS,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,...,140105.0,140106.0,140107.0,140108.0,140109.0,sum_all,avg_real,daste,cluster,final_cluster
ITEM_ITEM_ID,,,,,,,,,,,,,,,,,,,,,
632927.0,0,0.0,0,0,0,0,0.0,1,0.0,0,...,0,0,0,0,0.0,3.0,0.066667,-inf,0,-inf_0
639349.0,0,0.0,0,0,0,0,0.0,0,1.0,0,...,0,0,0,0,0.0,3.0,0.066667,-inf,0,-inf_0
640144.0,1,0.0,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0.0,3.0,0.066667,-inf,0,-inf_0
642304.0,0,0.0,0,0,0,0,0.0,0,1.0,0,...,0,0,0,0,0.0,3.0,0.066667,-inf,0,-inf_0
642900.0,0,1.0,0,0,0,0,0.0,0,0.0,0,...,0,0,0,0,0.0,3.0,0.066667,-inf,1,-inf_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766550.0,0,15.0,0,0,10,0,20.0,0,30.0,0,...,0,0,10,0,0.0,246.0,5.466667,225.5,0,225.5_0
1774398.0,0,1.0,12,48,35,35,65.0,38,106.0,26,...,148,12,29,51,65.0,2645.0,58.777778,225.5,0,225.5_0
1784784.0,0,14.0,0,60,30,0,20.0,0,12.0,0,...,0,16,0,0,20.0,421.0,9.355556,225.5,0,225.5_0


In [11]:
cluster_groupby = result_kmeans.groupby(['final_cluster']).agg(np.mean).iloc[:,:-4]
cluster_groupby2 = cluster_groupby.copy()
sum_cl = result_kmeans.groupby(['final_cluster']).agg(np.sum)
cluster_groupby['sumall'] = list(sum_cl['sum_all'].values)
cluster_groupby['avg_cluster'] = cluster_groupby2.mean(axis = 1)
cluster_groupby

NUM_MONTH_USE_SPMUS,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,...,140102.0,140103.0,140104.0,140105.0,140106.0,140107.0,140108.0,140109.0,sumall,avg_cluster
final_cluster,,,,,,,,,,,,,,,,,,,,,
-inf_0,0.072727,0.054545,0.063636,0.054545,0.100000,0.081818,0.100000,0.100000,0.127273,0.127273,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,330.00,0.066667
-inf_1,0.000000,0.076923,0.000000,0.000000,0.000000,0.230769,0.153846,0.076923,0.230769,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.00,0.066667
11.5_0,0.214511,0.287066,0.217666,0.378549,0.293375,0.315457,0.321767,0.233438,0.384858,0.343849,...,0.214511,0.271293,0.201893,0.331230,0.205047,0.220820,0.293375,0.242902,3953.00,0.277112
11.5_1,0.199346,0.323529,0.310458,0.398693,0.218954,0.388889,0.281046,0.267974,0.356209,0.431373,...,0.284314,0.375817,0.232026,0.441176,0.323529,0.267974,0.369281,0.447712,4419.00,0.320915
15.5_0,0.273210,0.519894,0.456233,0.527851,0.527851,0.419098,0.623342,0.408488,0.562334,0.379310,...,0.371353,0.445623,0.299735,0.490716,0.411141,0.376658,0.490716,0.572944,7709.00,0.454406
15.5_1,0.328395,0.279012,0.409877,0.496296,0.409877,0.271605,0.422222,0.296296,0.422222,0.419753,...,0.316049,0.456790,0.311111,0.320988,0.320988,0.348148,0.446914,0.372840,6881.00,0.377558
22.5_0,0.511327,0.682848,0.611650,0.731392,0.724919,0.647249,0.760518,0.614887,0.699029,0.692557,...,0.621359,0.692557,0.423948,0.708738,0.741100,0.511327,0.741100,0.682848,9213.00,0.662567
22.5_1,0.566986,0.677033,0.476077,0.636364,0.492823,0.490431,0.507177,0.547847,0.700957,0.710526,...,0.626794,0.574163,0.358852,0.610048,0.523923,0.485646,0.624402,0.662679,10356.00,0.550558
225.5_0,14.572852,34.685056,27.815691,24.372354,26.799502,26.549191,37.739103,30.475716,30.060399,32.821918,...,28.092154,26.801993,22.677460,39.774595,29.900374,23.914072,31.133250,31.136239,980433.66,27.132521


## ARIMA MODEL

In [12]:
from statsmodels.tsa.arima.model import ARIMA
forecast_result = []
for i in range(cluster_groupby.shape[0]): #22
    serie = cluster_groupby.iloc[i,:-2] # row of i
    model_arima = ARIMA(serie ,order = (1,0,0)) #create object from ARIMA class 
    result_arima = model_arima.fit() #fit the model with input parametters
    output = result_arima.forecast(12) # forecast 12 month further
    forecast_result.append(output) 
    
    #result_arima.summary() if you need to see parametters of ARIMA model run this in a new cell.

C:\Users\bgc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\bgc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\bgc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\bgc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\bgc\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g

In [13]:
predicted_df =  pd.DataFrame(data = np.array(forecast_result), columns=[('month'+str(i+1)) for i in range(12)],
                            index = cluster_groupby.index)
predicted_df

,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
final_cluster,,,,,,,,,,,,
-inf_0,0.035290,0.051580,0.059101,0.062572,0.064175,0.064915,0.065256,0.065414,0.065487,0.065520,0.065536,0.065543
-inf_1,0.069019,0.066687,0.066766,0.066763,0.066764,0.066763,0.066763,0.066763,0.066763,0.066763,0.066763,0.066763
11.5_0,0.277110,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109
11.5_1,0.322308,0.320926,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911
15.5_0,0.464256,0.455114,0.454344,0.454280,0.454274,0.454274,0.454274,0.454274,0.454274,0.454274,0.454274,0.454274
15.5_1,0.376281,0.377023,0.377184,0.377218,0.377226,0.377227,0.377228,0.377228,0.377228,0.377228,0.377228,0.377228
22.5_0,0.668537,0.663635,0.661955,0.661380,0.661183,0.661115,0.661092,0.661084,0.661081,0.661080,0.661080,0.661080
22.5_1,0.583318,0.560694,0.554244,0.552405,0.551881,0.551732,0.551689,0.551677,0.551674,0.551673,0.551672,0.551672
225.5_0,27.585365,27.163858,27.113823,27.107884,27.107179,27.107095,27.107085,27.107084,27.107084,27.107084,27.107084,27.107084


In [14]:
cluster_predict = pd.concat([cluster_groupby,predicted_df],axis =1 )
pd.set_option('display.max_columns', None)
cluster_predict.style.set_sticky(axis="index")
cluster_predict

,139801.0,139802.0,139803.0,139804.0,139805.0,139806.0,139807.0,139808.0,139809.0,139810.0,139811.0,139812.0,139901.0,139902.0,139903.0,139904.0,139905.0,139906.0,139907.0,139908.0,139909.0,139910.0,139911.0,139912.0,140001.0,140002.0,140003.0,140004.0,140005.0,140006.0,140007.0,140008.0,140009.0,140010.0,140011.0,140012.0,140101.0,140102.0,140103.0,140104.0,140105.0,140106.0,140107.0,140108.0,140109.0,sumall,avg_cluster,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
final_cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-inf_0,0.072727,0.054545,0.063636,0.054545,0.100000,0.081818,0.100000,0.100000,0.127273,0.127273,0.054545,0.063636,0.027273,0.072727,0.090909,0.100000,0.081818,0.090909,0.027273,0.063636,0.145455,0.072727,0.145455,0.081818,0.063636,0.045455,0.063636,0.127273,0.054545,0.118182,0.109091,0.127273,0.072727,0.109091,0.000000,0.109091,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,330.00,0.066667,0.035290,0.051580,0.059101,0.062572,0.064175,0.064915,0.065256,0.065414,0.065487,0.065520,0.065536,0.065543
-inf_1,0.000000,0.076923,0.000000,0.000000,0.000000,0.230769,0.153846,0.076923,0.230769,0.076923,0.153846,0.000000,0.076923,0.230769,0.000000,0.076923,0.076923,0.000000,0.076923,0.000000,0.076923,0.153846,0.230769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.00,0.066667,0.069019,0.066687,0.066766,0.066763,0.066764,0.066763,0.066763,0.066763,0.066763,0.066763,0.066763,0.066763
11.5_0,0.214511,0.287066,0.217666,0.378549,0.293375,0.315457,0.321767,0.233438,0.384858,0.343849,0.287066,0.205047,0.094637,0.280757,0.299685,0.340694,0.220820,0.365931,0.239748,0.274448,0.309148,0.223975,0.287066,0.359621,0.277603,0.201893,0.261830,0.328076,0.223975,0.239748,0.312303,0.309148,0.337539,0.315457,0.305994,0.343849,0.252366,0.214511,0.271293,0.201893,0.331230,0.205047,0.220820,0.293375,0.242902,3953.00,0.277112,0.277110,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109,0.277109
11.5_1,0.199346,0.323529,0.310458,0.398693,0.218954,0.388889,0.281046,0.267974,0.356209,0.431373,0.264706,0.189542,0.114379,0.316993,0.379085,0.303922,0.284314,0.352941,0.267974,0.339869,0.333333,0.392157,0.382353,0.339869,0.310458,0.323529,0.424837,0.359477,0.150327,0.310458,0.258170,0.450980,0.330065,0.392157,0.375817,0.349673,0.225490,0.284314,0.375817,0.232026,0.441176,0.323529,0.267974,0.369281,0.447712,4419.00,0.320915,0.322308,0.320926,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911,0.320911
15.5_0,0.273210,0.519894,0.456233,0.527851,0.527851,0.419098,0.623342,0.408488,0.562334,0.379310,0.466844,0.286472,0.135279,0.376658,0.448276,0.535809,0.374005,0.511936,0.376658,0.381963,0.371353,0.498674,0.503979,0.472149,0.461538,0.318302,0.631300,0.612732,0.493369,0.445623,0.374005,0.615385,0.554377,0.689655,0.429708,0.636605,0.289125,0.371353,0.445623,0.299735,0.490716,0.411141,0.376658,0.490716,0.572944,7709.00,0.454406,0.464256,0.455114,0.454344,0.454280,0.454274,0.454274,0.454274,0.454274,0.454274,0.454274,0.454274,0.454274
15.5_1,0.328395,0.279012,0.409877,0.496296,0.409877,0.271605,0.422222,0.296296,0.422222,0.419753,0.422222,0.212346,0.098765,0.293827,0.407407,0.375309,0.353086,0.370370,0.325926,0.409877,0.454321,0.333333,0.498765,0.444444,0.367901,0.259259,0.471605,0.432099,0.288889,0.498765,0.412346,0.513580,0.449383,0.508642,0.424691,0.409877,0.303704,0.316049,0.456790,0.311111,0.320988,0.320988,0.348148,0.446914,0.372840,6881.00,0.377558,0.376281,0.377023,0.377184,0.377218,0.377226,0.377227,0.377228,0.377228,0.377228,0.377228,0.377228,0.377228
22.5_0,0.511327,0.682848,0.611650,0.731392,0.724919,0.647249,0.760518,0.614887,0.699029,0.692557,0.689320,0.339806,0.165049,0.511327,0.524272,0.676375,0.417476,0.624595,0.744337,0.533981,0.624595,

In [15]:
ratio_total = []
for i in range(result_kmeans.shape[0]):
    item_cluster =result_kmeans['daste'].values[i]
    item_avg_real =result_kmeans['avg_real'].values[i]
    ratio_item = np.mean([])
    
    


C:\Users\bgc\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\bgc\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
np.mean([1,2])

1.5